In [1]:
import malariagen_data


In [2]:
ag3 = malariagen_data.Ag3(pre=True)

ag3.biallelic_snps_to_plink(results_dir='/Users/dennistpw/Projects/malariagen-data-python/',
                            region='2L:100000-2000000',
                            n_snps=2000,
                            sample_sets='AG1000G-AO',
                            )

In [9]:
region='2L:1001000-2009000',


In [58]:
ds = ag3.snp_calls(
    region=region,
    sample_sets=sample_sets
)

In [59]:
# Perform an allele count.
ac = ag3.snp_allele_counts(
    region=region,
    sample_sets=sample_sets
)

# Locate biallelic SNPs.
loc_bi = allel.AlleleCountsArray(ac).is_biallelic()

# Remap alleles to squeeze out unobserved alleles.
ac_bi = ac[loc_bi]
allele_mapping = trim_alleles(ac_bi)

Compute SNP allele counts:   0%|          | 0/30 [00:00<?, ?it/s]

In [66]:
ds_bi = ds.isel(variants=loc_bi)


In [94]:
ds_bi['variant_allele'].compute()

<xarray.DataArray 'variant_allele' (variants: 120913, alleles: 4)>
array([[b'G', b'A', b'C', b'T'],
       [b'A', b'C', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       ...,
       [b'G', b'A', b'C', b'T'],
       [b'C', b'A', b'T', b'G'],
       [b'T', b'A', b'C', b'G']], dtype='|S1')
Coordinates:
    variant_position  (variants) int32 100092 100095 100109 ... 1999872 1999929
    variant_contig    (variants) uint8 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: variants, alleles

In [67]:
from typing import Any, Dict, List, Optional, Tuple, Union


In [68]:
# Start building a new dataset.
coords: Dict[str, Any] = dict()
data_vars: Dict[str, Any] = dict()

In [69]:
# Store sample IDs.
coords["sample_id"] = ("samples",), ds_bi["sample_id"].data

# Store contig.
coords["variant_contig"] = ("variants",), ds_bi["variant_contig"].data

# Store position.
coords["variant_position"] = ("variants",), ds_bi["variant_position"].data

In [90]:
import dask.array as da
variant_allele = ds_bi["variant_allele"].data
variant_allele = variant_allele.rechunk((variant_allele.chunks[0], -1))
variant_allele_out = da.map_blocks(
    lambda block: apply_allele_mapping(block, allele_mapping, max_allele=1),
    variant_allele,
    dtype=variant_allele.dtype,
    chunks=(variant_allele.chunks[0], [2]),
)
variant_allele_out.compute()
# Store allele counts, transformed, so we don't have to recompute.
#ac_out = apply_allele_mapping(ac_bi, allele_mapping, max_allele=1)
#data_vars["variant_allele_count"] = ("variants", "alleles"), ac_out

AssertionError: 

When we try to select only biallelic snps, we
- count the number of alleles
- select only

In [95]:
allele_mapping

array([[ 0,  1, -1, -1],
       [ 0, -1, -1,  1],
       [ 0,  1, -1, -1],
       ...,
       [ 0, -1, -1,  1],
       [ 0, -1,  1, -1],
       [ 0, -1,  1, -1]], dtype=int32)

In [92]:
variant_allele.shape[0]

120913